# Extract GDP data

Extract the GDP data one line at a time (--> use cursor). Then transform that line of data and load the results into a SQLite database.

1.  connects to a SQLite database (worldbank.db) and creates a table to hold the gdp data.

In [5]:
#initial set up

import sqlite3

# connect to the database
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()


# create the test table including project_id as a primary key
cur.execute("DROP TABLE IF EXISTS gdp")
cur.execute("CREATE TABLE gdp (countryname TEXT, countrycode TEXT, year INTEGER, gdp REAL, PRIMARY KEY (countrycode, year));")

conn.commit()
conn.close()

In [6]:
#read in a data file one line at a time, run a transformation on that row of data, and then move on to the next row in the file.

def extract_lines(file):
    while True:
        line = file.readline()
        if not line:
            break
        yield line

In [7]:
import pandas as pd
import numpy as np
import sqlite3

#transforms the data in preparation for a load step.

# transform the indicator data
def transform_indicator_data(data, colnames):
    
    # get rid of quote marks
    for i, datum in enumerate(data):
        data[i] = datum.replace('"','')
    
    country = data[0]
    
    # filter out values that are not countries
    non_countries = ['World',
     'High income',
     'OECD members',
     'Post-demographic dividend',
     'IDA & IBRD total',
     'Low & middle income',
     'Middle income',
     'IBRD only',
     'East Asia & Pacific',
     'Europe & Central Asia',
     'North America',
     'Upper middle income',
     'Late-demographic dividend',
     'European Union',
     'East Asia & Pacific (excluding high income)',
     'East Asia & Pacific (IDA & IBRD countries)',
     'Euro area',
     'Early-demographic dividend',
     'Lower middle income',
     'Latin America & Caribbean',
     'Latin America & the Caribbean (IDA & IBRD countries)',
     'Latin America & Caribbean (excluding high income)',
     'Europe & Central Asia (IDA & IBRD countries)',
     'Middle East & North Africa',
     'Europe & Central Asia (excluding high income)',
     'South Asia (IDA & IBRD)',
     'South Asia',
     'Arab World',
     'IDA total',
     'Sub-Saharan Africa',
     'Sub-Saharan Africa (IDA & IBRD countries)',
     'Sub-Saharan Africa (excluding high income)',
     'Middle East & North Africa (excluding high income)',
     'Middle East & North Africa (IDA & IBRD countries)',
     'Central Europe and the Baltics',
     'Pre-demographic dividend',
     'IDA only',
     'Least developed countries: UN classification',
     'IDA blend',
     'Fragile and conflict affected situations',
     'Heavily indebted poor countries (HIPC)',
     'Low income',
     'Small states',
     'Other small states',
     'Not classified',
     'Caribbean small states',
     'Pacific island small states']
    
    if country not in non_countries:
        data_array = np.array(data, ndmin=2)
        data_array.reshape(1, 63)
        df = pd.DataFrame(data_array, columns=colnames).replace('', np.nan)
        df.drop(['\n', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)

        # Reshape the data sets so that they are in long format
        df_melt = df.melt(id_vars=['Country Name', 'Country Code'], 
                            var_name='year', 
                            value_name='gdp')
        
        results = []
        for index, row in df_melt.iterrows():
            country, countrycode, year, gdp = row
            if str(gdp) != 'nan':
                results.append([country, countrycode, year, gdp])
        return results

In [17]:
#load the trasnformed data into the gdp table in the worldbank.db database.

def load_indicator_data(results):
    conn = sqlite3.connect('worldbank.db')
    cur = conn.cursor()
    if results:
        for result in results:
            countryname, countrycode, year, gdp = result

            sql_string = 'INSERT INTO gdp (countryname, countrycode, year, gdp) VALUES ("{}", "{}", {}, {});'.format(countryname, countrycode, year, gdp)

            # connect to database and execute query
            try:
                cur.execute(sql_string)
            except Exception as e:
                print('error occurred:', e, result)
            
    conn.commit()
    conn.close()
    
    return None

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
#run the ETL pipeline
with open('drive/My Drive/gdp_data.csv') as f:
    for line in extract_lines(f):
        data = line.split(',')
        if len(data) == 63:
            if data[0] == '"Country Name"':
                colnames = []
                # get rid of quote marks
                for i, datum in enumerate(data):
                    colnames.append(datum.replace('"',''))
            else:
                # transform and load the line of indicator data
                results = transform_indicator_data(data, colnames)
                load_indicator_data(results)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1992', '6976080330.65779']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1993', '6126456175.15147']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1994', '6294803496.77744']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1995', '7018100153.49728']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1996', '7330965239.09774']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1997', '7548912105.47174']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1998', '8453704212.19223']
error occurred: UNIQUE constraint failed: gdp.countrycode, gdp.year ['Iceland', 'ISL', '1999', '8917286036.00179']
error occurred: UNIQUE constraint failed: gd

In [19]:
conn = sqlite3.connect('worldbank.db')

# get a cursor
cur = conn.cursor()

# create the test table including project_id as a primary key
df = pd.read_sql("SELECT * FROM gdp", con=conn)

conn.commit()
conn.close()

df

,countryname,countrycode,year,gdp
0,Aruba,ABW,1994,1.330168e+09
1,Aruba,ABW,1995,1.320670e+09
2,Aruba,ABW,1996,1.379888e+09
3,Aruba,ABW,1997,1.531844e+09
4,Aruba,ABW,1998,1.665363e+09
...,...,...,...,...
8709,Zimbabwe,ZWE,2013,1.545177e+10
8710,Zimbabwe,ZWE,2014,1.589105e+10
8711,Zimbabwe,ZWE,2015,1.630467e+10
8712,Zimbabwe,ZWE,2016,1.661996e+10
